In [ ]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import lightgbm
from lightgbm import LGBMClassifier
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from multiprocessing import Process# this is used for multithreading
import multiprocessing
import codecs# this is used for file operations 
import random as r
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
np.random.seed(1)

In [2]:
def reduce_mem_usage(df):
    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 
                     'MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) 
                                             / start_mem))
    
    return df

In [4]:
train_df = reduce_mem_usage(pd.read_csv("Featured_train.csv"))

Memory usage of dataframe is 4086.12MB
Memory usage after optimization is: 988.93MB
Decreased by 75.8%


In [ ]:
y_true = train_df['event']
train_df.drop(columns=['event'],axis=1,inplace=True)

In [ ]:
data = train_df[:1]

In [77]:
def function1(X):
     #converting data into numpy array
    if(type(X)=='pandas.core.frame.DataFrame'):
        X = X.to_numpy()
    else:
        X = np.array(X)
        
    if(X.shape[1]!=27):
        return "27 featurs are required for prediction"
    experiment_encode= {'CA':0,'DA':1,'SS':2,'LOFT':3}
    X[0][1] =experiment_encode[X[0][1]]
    #loading Best Model
    model = pickle.load(open("LightGBM_best.sav","rb"))
    event_encode= {0:'A',1:'B',2:'C',3:'D'}
    return event_encode[np.argmax(model.predict(X))]
result =  final(data)
print("Model has predicted Event : ",result)

Model has predicted Event :  A


In [24]:
from sklearn.metrics import accuracy_score,log_loss,confusion_matrix
def function2(X,y):
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
  event_encode= {'A':0,'B':1,'C':2,'D':3}
  y = y.apply(lambda x: event_encode[x])
  experiment_encode= {'CA':0,'DA':1,'SS':2,'LOFT':3}
  X['experiment'] = X['experiment'].apply(lambda x: experiment_encode[x])
  X['experiment'] = X['experiment'].astype('int8')
  model = pickle.load(open("LightGBM_best.sav","rb"))
  predict_y_proba = model.predict(X)
  predict_y  = np.argmax(predict_y_proba,axis=1)
  print("Accurcy score of model",accuracy_score(y,predict_y))
  print("Log loss of  Model",log_loss(y, predict_y_proba))
  print(confusion_matrix(y,predict_y))
    
function2(train_df,y_true)   

Accurcy score of model 0.9505395156901365
Log loss of  Model 0.17548799084282016
[[2841552     826    2024    4407]
 [  49159   81438       0       0]
 [    443       0 1652243       0]
 [ 183886       0       0   51443]]
